In [1]:
from snli.bert.bert import *
from snli.train_utils import *

In [2]:
from transformers import DistilBertTokenizer, DistilBertModel
encoder = DistilBertModel.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

data_module = snli_bert_data_module(32,combine=True,tokenizer = tokenizer)
Lang = data_module.Lang
embedding_matrix = None


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import BertModel,DistilBertModel


class Bert_Encoder_conf:
    encoder_dim = 768
    encoder = None
    batch_size = None
    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

class Bert_Encoder(nn.Module):
    def __init__(self,conf):
        super(Bert_Encoder,self).__init__()
        self.conf=conf
        self.bert = conf.encoder
        self.fc = nn.Linear(conf.encoder_dim,3)

    def forward(self,x0):
        enc = self.bert.forward(x0)[0][:, 0, :]
        opt = self.fc(enc)
        opt = opt.unsqueeze(0)
        return opt

In [9]:
# model_conf = Bert_Encoder_conf()
model_conf = Bert_Encoder_conf(**{"encoder":encoder})
model = Bert_Encoder(model_conf)
model.cuda()


Bert_Encoder(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featur

In [10]:

data_module.batch_size = 256
for batch in data_module.val_dataloader():
    inp = batch[0]
    print(model(inp.cuda()))
    break

tensor([[[-1.5606e-01,  3.3819e-01, -5.2539e-02],
         [-1.5597e-01,  3.1152e-01, -8.4443e-02],
         [-1.6710e-01,  2.8999e-01, -9.2950e-02],
         [-1.3891e-01,  2.8695e-01,  2.7773e-02],
         [-1.5344e-01,  2.7768e-01,  3.5789e-02],
         [-1.5021e-01,  2.6954e-01,  3.5361e-02],
         [-8.6597e-02,  2.8779e-01, -3.1927e-03],
         [-1.0897e-01,  3.0146e-01,  2.2138e-02],
         [-1.0828e-01,  2.7488e-01, -6.9044e-03],
         [-1.1929e-01,  2.9659e-01, -7.0752e-02],
         [-1.0563e-01,  2.8370e-01, -6.9148e-02],
         [-1.0763e-01,  2.8029e-01, -7.3718e-02],
         [-1.3063e-01,  2.9325e-01, -4.3215e-02],
         [-1.4046e-01,  3.0330e-01, -3.2987e-02],
         [-1.3897e-01,  3.0665e-01, -3.6979e-02],
         [-1.7560e-01,  2.7722e-01,  6.6635e-02],
         [-1.5968e-01,  2.7369e-01,  6.9692e-02],
         [-1.4553e-01,  2.6856e-01,  3.4076e-02],
         [-1.1465e-01,  2.5734e-01, -1.0196e-02],
         [-1.1534e-01,  2.6951e-01, -2.9981e-02],


In [13]:
encoder.forward(inp.cuda())[0][:, 0, :].shape

torch.Size([256, 512])

In [14]:
from transformers import BertTokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
from transformers import BertModel,DistilBertModel
import glob
import re
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from snli.bilstm.bilstm import *
from snli.attn_enc.attn_enc import *
from lang import *
import joblib


In [5]:
def load_bert_encoder():
    BERT_ENC_PATH = f"./models/bert_encoder/"
    with open(BERT_ENC_PATH + "model_conf.pkl", "rb") as f:
        model_conf = pickle.load(f)
    with open(BERT_ENC_PATH + "lang.pkl", "rb") as f:
        Lang = joblib.load(f)
    encoder = DistilBertModel.from_pretrained("distilbert-base-uncased")
    encoder.load_state_dict(torch.load(BERT_ENC_PATH + "weights.pt"))
    return encoder, Lang

In [6]:
load_bert_encoder()

(DistilBertModel(
   (embeddings): Embeddings(
     (word_embeddings): Embedding(30522, 768, padding_idx=0)
     (position_embeddings): Embedding(512, 768)
     (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     (dropout): Dropout(p=0.1, inplace=False)
   )
   (transformer): Transformer(
     (layer): ModuleList(
       (0): TransformerBlock(
         (attention): MultiHeadSelfAttention(
           (dropout): Dropout(p=0.1, inplace=False)
           (q_lin): Linear(in_features=768, out_features=768, bias=True)
           (k_lin): Linear(in_features=768, out_features=768, bias=True)
           (v_lin): Linear(in_features=768, out_features=768, bias=True)
           (out_lin): Linear(in_features=768, out_features=768, bias=True)
         )
         (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
         (ffn): FFN(
           (dropout): Dropout(p=0.1, inplace=False)
           (lin1): Linear(in_features=768, out_features=3072, bias=True)
     

In [1]:
from novelty_serve import *

In [2]:
download_from_drive()

In [1]:

import joblib
import pickle
import argparse
from lang import *
from novelty.cnn.cnn_model import *
from snli.bilstm.bilstm import *
from snli.attn_enc.attn_enc import *
from joblib import Memory
import shutil
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
from pytorch_lightning.metrics import Accuracy
from utils import load_bilstm_encoder, load_attn_encoder,seed_torch
from novelty.train_utils import *
from datamodule import *
import os
from keys import NEPTUNE_API

seed_torch()


encoder_ = "BiLSTM"
dataset_ = "DLND"

if encoder_ == 'Attention':
    encoder, Lang = load_attn_encoder()
else:
    encoder, Lang = load_bilstm_encoder()

if dataset_ == "Webis":
    data_module = webis_data_module(Lang)
else:
    data_module = dlnd_data_module(Lang)

params = {
    "num_filters": 60,
    "dropout": 0.3,
    "expand features": True,
    "filter_sizes": [4, 6, 9],
    "freeze_embedding": False,
    "activation": "tanh",
    "optim": "adamw",
    "weight_decay":0.1,
    "lr": 0.00010869262115700171,
    "scheduler": "constant",
}

model_conf = Novelty_CNN_conf(100, encoder, **params)
model = Novelty_CNN_model(DeepNoveltyCNN, model_conf, params)

EPOCHS = 5

neptune_logger = NeptuneLogger(
    api_key=NEPTUNE_API,
    project_name="aditya140/NoveltyCNN",
    experiment_name="Evaluation",  # Optional,
    tags=[ dataset_, "test", "CNN",encoder_],
)

tensorboard_logger = TensorBoardLogger("lightning_logs")

lr_logger = LearningRateLogger(logging_interval="step")
neptune_logger.experiment.log_metric("epochs", EPOCHS)
trainer = pl.Trainer(
    gpus=1,
    max_epochs=EPOCHS,
    progress_bar_refresh_rate=10,
    profiler=False,
    auto_lr_find=False,
    callbacks=[lr_logger],
    logger=[neptune_logger, tensorboard_logger],
    row_log_interval=2,
)
trainer.fit(model, data_module)
trainer.test(model, datamodule=data_module)

MODEL_PATH = "./models/cnn_novelty/"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
torch.save(model.model.state_dict(), MODEL_PATH + "weights.pt")
with open(MODEL_PATH + "model_conf.pkl", "wb") as f:
    pickle.dump(model_conf, f)
with open(MODEL_PATH + "lang.pkl", "wb") as f:
    pickle.dump(Lang, f)
shutil.make_archive("./models/cnn_novelty", "zip", "./models/cnn_novelty")
neptune_logger.experiment.log_artifact("./models/cnn_novelty.zip")


https://ui.neptune.ai/aditya140/NoveltyCNN/e/NOVELTY-319
NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type           | Params
-----------------------------------------
0 | model | DeepNoveltyCNN | 19 M  


Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVELTY-319. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: afa9fa66-c607-46ad-b537-c4c061f

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVELTY-319. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: afa9fa66-c607-46ad-b537-c4c061f

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVELTY-319. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: afa9fa66-c607-46ad-b537-c4c061f

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVELTY-319. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: afa9fa66-c607-46ad-b537-c4c061f

Saving latest checkpoint..

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVELTY-319. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: afa

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9078),
 'test_f1': tensor(0.9057),
 'test_loss': tensor(0.2386, device='cuda:0')}
--------------------------------------------------------------------------------



In [38]:
true = torch.zeros((0))
pred = torch.zeros((0,2))
for i in data_module.test_dataloader():
    a,b,c = i
    true = torch.cat([true,c],dim = 0)
    opt = model(a.cuda(),b.cuda()).detach().cpu()
    pred = torch.cat([pred,opt],dim = 0)


In [39]:
y_score = pred.numpy()
Y_test = true.numpy()
Y_test = np.append((1-np.expand_dims(Y_test, axis=0)),np.expand_dims(Y_test, axis=0),0).transpose()

In [6]:
from utils import load_bilstm_encoder, load_attn_encoder,seed_torch

encoder, Lang = load_bilstm_encoder()


In [7]:
from datamodule import IMDBDataModule

In [3]:
data = IMDBDataModule()

In [4]:
data.prepare_data(Lang,100)

50000


In [8]:
for i in data.train_dataloader():
    print(i)
    break

[tensor([[[  101,  3040,  2933,  ...,     0,     0,     0],
         [  101,  2028,  1997,  ...,     0,     0,     0],
         [  101,  1996, 12700,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[  101,  2023,  5015,  ...,     0,     0,     0],
         [  101,  2009,  2428,  ...,     0,     0,     0],
         [  101,  2009,  2001,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[  101,  2065,  2017,  ...,     0,     0,     0],
         [  101,  2025,  3243,  ...,     0,     0,     0],
         [  101,  1037,  2442,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         

In [3]:
import sys

sys.path.append(".")
import joblib
import pickle
import argparse
from lang import *
from document.han.han import *
from snli.bilstm.bilstm import *
from snli.attn_enc.attn_enc import *
from joblib import Memory
import shutil
import pytorch_lightning as pl
from document.han.han import HAN_conf,HAN_classifier,HAN
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
from pytorch_lightning.metrics import Accuracy
from utils import load_bilstm_encoder, load_attn_encoder
from document.train_utils import *
from datamodule import *
import os
from keys import NEPTUNE_API


encoder_ = 'bilstm'


if encoder_ == "bilstm":
    encoder, Lang = load_bilstm_encoder()
elif encoder_ == "attention":
    encoder, Lang = load_attn_encoder()

data_module = imdb_data_module(Lang)

params = {
    "optim": "adamw",
    "weight_decay":0.1,
    "lr": 0.00010869262115700171,
    "scheduler": "constant",
}

model_conf = HAN_conf(100, encoder, **params)
model = Document_model_clf(HAN_classifier, model_conf, params)

EPOCHS = 3

neptune_logger = NeptuneLogger(
    api_key=NEPTUNE_API,
    project_name="aditya140/Imdb",
    experiment_name="training", 
    tags=['HAN'],
)

tensorboard_logger = TensorBoardLogger("lightning_logs")

lr_logger = LearningRateLogger(logging_interval="step")
neptune_logger.experiment.log_metric("epochs", EPOCHS)
trainer = pl.Trainer(
    gpus=1,
    max_epochs=EPOCHS,
    progress_bar_refresh_rate=10,
    profiler=False,
    auto_lr_find=False,
    callbacks=[lr_logger],
    logger=[neptune_logger, tensorboard_logger],
    row_log_interval=2,
)
trainer.fit(model, data_module)
trainer.test(model, datamodule=data_module)


MODEL_PATH = "./models/document_imdb_han/"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
torch.save(model.model.state_dict(), MODEL_PATH + "weights.pt")
with open(MODEL_PATH + "model_conf.pkl", "wb") as f:
    pickle.dump(model_conf, f)
with open(MODEL_PATH + "lang.pkl", "wb") as f:
    pickle.dump(Lang, f)
shutil.make_archive("./models/document_imdb_han", "zip", "./models/document_imdb_han")
neptune_logger.experiment.log_artifact("./models/document_imdb_han.zip")


https://ui.neptune.ai/aditya140/Imdb/e/IM-11
NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type           | Params
-----------------------------------------
0 | model | HAN_classifier | 19 M  


Saving latest checkpoint..



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4235),
 'test_f1': tensor(0.2145),
 'test_loss': tensor(1.5252, device='cuda:0')}
--------------------------------------------------------------------------------



In [1]:
import sys

sys.path.append(".")
import joblib
import pickle
import argparse
from lang import *
from document.han.han import *
from snli.bilstm.bilstm import *
from snli.attn_enc.attn_enc import *
from joblib import Memory
import shutil
import pytorch_lightning as pl
from document.han.han import HAN_conf,HAN_classifier,HAN
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
from pytorch_lightning.metrics import Accuracy
from utils import load_bilstm_encoder, load_attn_encoder
from document.train_utils import *
from datamodule import *
import os
from keys import NEPTUNE_API


encoder_ = 'bilstm'


if encoder_ == "bilstm":
    encoder, Lang = load_bilstm_encoder()
elif encoder_ == "attention":
    encoder, Lang = load_attn_encoder()

data_module = imdb_data_module(Lang)

params = {
    "optim": "adamw",
    "weight_decay":0.1,
    "lr": 0.00010869262115700171,
    "scheduler": "constant",
}

model_conf = HAN_conf(100, encoder, **params)
model = Document_model_reg(HAN_regressor, model_conf, params)

EPOCHS = 5

neptune_logger = NeptuneLogger(
    api_key=NEPTUNE_API,
    project_name="aditya140/Imdb",
    experiment_name="training", 
    tags=['HAN'],
)

tensorboard_logger = TensorBoardLogger("lightning_logs")

lr_logger = LearningRateLogger(logging_interval="step")
neptune_logger.experiment.log_metric("epochs", EPOCHS)
trainer = pl.Trainer(
    gpus=1,
    max_epochs=EPOCHS,
    progress_bar_refresh_rate=10,
    profiler=False,
    auto_lr_find=False,
    callbacks=[lr_logger],
    logger=[neptune_logger, tensorboard_logger],
    row_log_interval=2,
)
trainer.fit(model, data_module)
trainer.test(model, datamodule=data_module)


MODEL_PATH = "./models/document_imdb_han/"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
torch.save(model.model.state_dict(), MODEL_PATH + "weights.pt")
with open(MODEL_PATH + "model_conf.pkl", "wb") as f:
    pickle.dump(model_conf, f)
with open(MODEL_PATH + "lang.pkl", "wb") as f:
    pickle.dump(Lang, f)
shutil.make_archive("./models/document_imdb_han", "zip", "./models/document_imdb_han")
neptune_logger.experiment.log_artifact("./models/document_imdb_han.zip")



https://ui.neptune.ai/aditya140/Imdb/e/IM-18
NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type          | Params
------------------------------------------
0 | model   | HAN_regressor | 19 M  
1 | loss_fn | MSELoss       | 0     


Saving latest checkpoint..



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.3101),
 'test_f1': tensor(0.2188),
 'test_loss': tensor(2.4614, device='cuda:0'),
 'test_rmse': tensor(1.5594, device='cuda:0')}
--------------------------------------------------------------------------------



In [2]:
for i in data_module.test_dataloader():
    print(i[1].dtype)
    break

torch.int64


In [3]:
a = model(i[0].cuda()).squeeze(1)

In [4]:
torch.round(a).to(torch.int64)

tensor([2, 1, 1, 5, 6, 6, 1, 6, 6, 6, 3, 7, 7, 1, 6, 7, 2, 5, 0, 3, 4, 4, 2, 6,
        2, 2, 6, 6, 1, 2, 1, 3, 1, 7, 5, 6, 5, 1, 2, 5, 6, 3, 0, 5, 1, 8, 6, 1,
        1, 4, 0, 4, 0, 1, 1, 7, 1, 7, 6, 3, 7, 4, 4, 1], device='cuda:0')

In [5]:
i[1]

tensor([5, 3, 0, 7, 3, 4, 2, 4, 4, 7, 5, 5, 7, 0, 7, 7, 6, 5, 0, 3, 6, 5, 4, 5,
        3, 3, 7, 6, 0, 3, 0, 5, 2, 7, 2, 7, 5, 2, 3, 6, 7, 3, 3, 7, 0, 7, 7, 0,
        3, 1, 0, 7, 0, 3, 1, 7, 0, 7, 7, 3, 6, 3, 3, 0])

In [12]:
metric = Accuracy(num_classes=10)
metricF1 = F1(num_classes=10)

In [13]:
metric(torch.round(a).to(torch.int64).detach().cpu(),i[1])

tensor(0.2031)

In [14]:
loss

tensor(15.6313, grad_fn=<MseLossBackward>)

In [2]:
from utils import *
han,lang = load_han_reg_encoder()

In [3]:
han

HAN(
  (encoder): Attn_Encoder(
    (embedding): Embedding(30522, 300, padding_idx=0)
    (translate): Linear(in_features=300, out_features=400, bias=True)
    (act): Tanh()
    (lstm_layer): LSTM(400, 400, num_layers=2, bidirectional=True)
    (attention): Attention(
      (Ws): Linear(in_features=800, out_features=100, bias=False)
      (Wa): Linear(in_features=100, out_features=1, bias=False)
    )
  )
  (translate): Linear(in_features=800, out_features=300, bias=True)
  (act): ReLU()
  (lstm_layer): LSTM(300, 300, num_layers=2, bidirectional=True)
  (attention): Attention(
    (Ws): Linear(in_features=600, out_features=50, bias=False)
    (Wa): Linear(in_features=50, out_features=1, bias=False)
  )
)

In [5]:
a = """The CNN then extracts the features from the relative document representation and finally classifies the incoming document. Results signify that our network is able to learn the complex semantic interactions between source and target information necessary to conclude upon the state of novelty of a document. It is to be noted that except (Ghosal et al., 2018) and our baselines, all other measures that we consider for comparison were developed with an information retrieval perspective. The p-values for F1 score produced by 20 runs of our system against the baseline are less than 0.05 and hence the improvement is statistically significant and unlikely to be observed by chance in 95 confidence interval."""


In [7]:
doc_vec = [lang.preprocess_sentence(i) for i in a.split(".")]

In [11]:
def to_novelty_tensor(s, Lang, num_sent):
    if isinstance(s, list):
        s = ". ".join(s)
    pad_arr = [0] * Lang.max_len
    t = Lang.encode_batch(Lang.preprocess_sentence(s).split("."))
    if t.shape[0] < num_sent:
        opt = np.append(t, [pad_arr] * (num_sent - t.shape[0]), axis=0)
    else:
        opt = t[
            : model_conf.num_sent,
        ]
    return opt

In [16]:
doc_ = torch.tensor(to_novelty_tensor(a,lang,100)).unsqueeze(0)
han(doc_).shape

torch.Size([1, 600])

In [18]:
a = han(doc_)

In [1]:
import sys

sys.path.append(".")
import joblib
import pickle
import argparse
from lang import *
from novelty.han.han_novelty import *
from snli.bilstm.bilstm import *
from snli.attn_enc.attn_enc import *
from joblib import Memory
import shutil
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
from pytorch_lightning.metrics import Accuracy
from utils import load_bilstm_encoder, load_attn_encoder, load_han_clf_encoder,load_han_reg_encoder,reset_model
from novelty.train_utils import *
from datamodule import *
import os
from keys import NEPTUNE_API

encoder_ ='reg'

if encoder_ == "reg":
    encoder, Lang = load_han_reg_encoder()
elif encoder_ == "clf":
    encoder, Lang = load_han_clf_encoder()

data_module = dlnd_data_module(Lang)

params = {
    "optim": "adamw",
    "weight_decay":0.1,
    "lr": 0.00010869262115700171,
    "scheduler": "lambda",
}

model_conf = HAN_Novelty_conf(encoder, **params)
model = Novelty_CNN_model(HAN_Novelty, model_conf, params)

model.model = reset_model(model.model)

EPOCHS = 5

neptune_logger = NeptuneLogger(
    api_key=NEPTUNE_API,
    project_name="aditya140/Novelty",
    experiment_name="Evaluation",  # Optional,
    tags=["DLND", "test", "HAN"],
)

tensorboard_logger = TensorBoardLogger("lightning_logs")

lr_logger = LearningRateLogger(logging_interval="step")
neptune_logger.experiment.log_metric("epochs", EPOCHS)
trainer = pl.Trainer(
    gpus=1,
    max_epochs=EPOCHS,
    progress_bar_refresh_rate=10,
    profiler=False,
    auto_lr_find=False,
    callbacks=[lr_logger],
    logger=[neptune_logger, tensorboard_logger],
    row_log_interval=2,
)
trainer.fit(model, data_module)
trainer.test(model, datamodule=data_module)

MODEL_PATH = "./models/cnn_novelty/"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
torch.save(model.model.state_dict(), MODEL_PATH + "weights.pt")
with open(MODEL_PATH + "model_conf.pkl", "wb") as f:
    pickle.dump(model_conf, f)
with open(MODEL_PATH + "lang.pkl", "wb") as f:
    pickle.dump(Lang, f)
shutil.make_archive("./models/cnn_novelty", "zip", "./models/cnn_novelty")
neptune_logger.experiment.log_artifact("./models/cnn_novelty.zip")


https://ui.neptune.ai/aditya140/Novelty/e/NOVE-223
NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | HAN_Novelty | 21 M  


Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVE-223. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: a9f80a4c-b813-440e-b911-d4d59ac342

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVE-223. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: a9f80a4c-b813-440e-b911-d4d59ac342

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVE-223. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: a9f80a4c-b813-440e-b911-d4d59ac342

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVE-223. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: a9f80a4c-b813-440e-b911-d4d59ac342

Saving latest checkpoint..

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment NOVE-223. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: a9f80a

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.8754),
 'test_f1': tensor(0.8691),
 'test_loss': tensor(0.4162, device='cuda:0')}
--------------------------------------------------------------------------------



# Structured Self Attention

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.nn.init import kaiming_uniform_
import math


class Struc_Attn_Encoder_conf:
    embedding_dim = 300
    hidden_size = 300
    fcs = 1
    r = 30
    num_layers = 2
    dropout = 0.1
    opt_labels = 3
    bidirectional = True
    attn_type = "dot"
    attention_layer_param = 100
    activation = "tanh"
    freeze_embedding = False
    gated_embedding_dim = 100
    gated = True
    pool_strategy = 'max' # max,avg
    penalty =True

    def __init__(self, lang, embedding_matrix=None, **kwargs):
        self.embedding_matrix = None
        if lang.tokenizer == "BERT":
            self.vocab_size = lang.vocab_size
            self.padding_idx = lang.bert_tokenizer.vocab["[PAD]"]
        else:
            self.embedding_matrix = embedding_matrix
            self.vocab_size = lang.vocab_size_final()
            self.padding_idx = lang.word2idx[lang.config.pad]
        for k, v in kwargs.items():
            setattr(self, k, v)


class Struc_Attention(nn.Module):
    def __init__(self, conf):
        super(Struc_Attention, self).__init__()
        self.Ws = nn.Linear(
            (2 if conf.bidirectional else 1) * conf.hidden_size,
            conf.attention_layer_param,
            bias=False,
        )
        self.Wa = nn.Linear(conf.attention_layer_param, conf.r, bias=False)

    def forward(self, hid):
        opt = self.Ws(hid)
        opt = F.tanh(opt)
        opt = self.Wa(opt)
        opt = F.softmax(opt)
        return opt


class Struc_Attn_Encoder(nn.Module):
    def __init__(self, conf):
        super(Struc_Attn_Encoder, self).__init__()
        self.conf = conf
        self.embedding = nn.Embedding(
            num_embeddings=self.conf.vocab_size,
            embedding_dim=self.conf.embedding_dim,
            padding_idx=self.conf.padding_idx,
        )
        self.translate = nn.Linear(self.conf.embedding_dim, self.conf.hidden_size) # make (300,..) if not working
        self.init_activation()
        if isinstance(self.conf.embedding_matrix, np.ndarray):
            self.embedding.from_pretrained(
                torch.tensor(self.conf.embedding_matrix),
                freeze=self.conf.freeze_embedding,
            )
        self.lstm_layer = nn.LSTM(
            input_size=self.conf.hidden_size,
            hidden_size=self.conf.hidden_size,
            num_layers=self.conf.num_layers,
            bidirectional=self.conf.bidirectional,
        )
        self.attention = Struc_Attention(conf)
    
    def init_activation(self):
        if self.conf.activation.lower() == "relu".lower():
            self.act = nn.ReLU()
        elif self.conf.activation.lower() == "tanh".lower():
            self.act = nn.Tanh()
        elif self.conf.activation.lower() == "leakyrelu".lower():
            self.act = nn.LeakyReLU()

    def forward(self, inp):
        batch_size = inp.shape[0]
        embedded = self.embedding(inp)
        embedded = self.translate(embedded)
        embedded = self.act(embedded)
        embedded = embedded.permute(1, 0, 2)
        all_, (hid, cell) = self.lstm_layer(embedded)
        attn = self.attention(all_)
        cont = torch.bmm(all_.permute(1, 2, 0), attn.permute(1, 0, 2)).permute(2, 0, 1)
        return cont,attn


class Struc_Attn_encoder_snli(nn.Module):
    def __init__(self, conf):
        super(Struc_Attn_encoder_snli, self).__init__()
        self.conf = conf
        self.encoder = Struc_Attn_Encoder(conf)
        self.gated = self.conf.gated
        self.penalty = self.conf.penalty
        self.template = nn.Parameter(torch.zeros((1)), requires_grad=True)

        # Gated parameters
        if self.gated :
            self.wt_p = torch.nn.Parameter(
            torch.rand(
                    (self.conf.r,
                    (2 if conf.bidirectional else 1) * self.conf.hidden_size,
                    self.conf.gated_embedding_dim
                    )
                ))
            self.wt_h = torch.nn.Parameter(
                torch.rand(
                        (self.conf.r,
                        (2 if conf.bidirectional else 1) * self.conf.hidden_size,
                        self.conf.gated_embedding_dim
                        )
                    ))
            self.fc_in = nn.Linear(
                self.conf.gated_embedding_dim * self.conf.r,
                self.conf.hidden_size,
            )
            self.fcs = nn.ModuleList(
                [
                    nn.Linear(self.conf.hidden_size, self.conf.hidden_size)
                    for i in range(self.conf.fcs)
                ]
            )
            self.fc_out = nn.Linear(self.conf.hidden_size, self.conf.opt_labels)

        # Non Gated Version (max pool avg pool)
        else:
            self.fc_in = nn.Linear(
            (2 if conf.bidirectional else 1) * 4 * self.conf.hidden_size,
            self.conf.hidden_size,
            )
            self.fcs = nn.ModuleList(
                [
                    nn.Linear(self.conf.hidden_size, self.conf.hidden_size)
                    for i in range(self.conf.fcs)
                ]
            )
            self.fc_out = nn.Linear(self.conf.hidden_size, self.conf.opt_labels)


        self.init_activation()
        self.softmax = nn.Softmax(dim=2)
        self.dropout = nn.Dropout(p=self.conf.dropout)
        self.init_gated_encoder()

    def init_gated_encoder(self):
        kaiming_uniform_(self.wt_p)
        kaiming_uniform_(self.wt_h)

    def init_activation(self):
        if self.conf.activation.lower() == "relu".lower():
            self.act = nn.ReLU()
        elif self.conf.activation.lower() == "tanh".lower():
            self.act = nn.Tanh()
        elif self.conf.activation.lower() == "leakyrelu".lower():
            self.act = nn.LeakyReLU()

    def penalty_l2(self,att):
        att = att.permute(1,0,2)
        penalty = (torch.norm(torch.bmm(att,att.transpose(1,2)) - torch.eye(att.size(1)).to(self.template.device),p='fro'))**2/att.size(0)
        return penalty

    def forward(self, x0, x1):
        x0_enc,x0_attn = self.encoder(x0.long())
        x0_enc = self.dropout(x0_enc)
        x1_enc,x1_attn = self.encoder(x1.long())
        x1_enc = self.dropout(x1_enc)

        if self.gated:
            F0 = x0_enc @ self.wt_p
            F1 = x1_enc @ self.wt_h
            
            Fr = F0*F1

            Fr = Fr.permute(1,0,2).flatten(start_dim=1)
        else:
            F0 = x0_enc.mean(0,keepdim=True)
            F1 = x1_enc.mean(0,keepdim=True)
            Fr = torch.cat(
                [F0, F1, torch.abs(F0 - F1), F0 * F1], dim=2
            )

        opt = self.fc_in(Fr)
        opt = self.dropout(opt)
        for fc in self.fcs:
            opt = fc(opt)
            opt = self.dropout(opt)
            opt = self.act(opt)
        opt = self.fc_out(opt)
        if self.penalty:
            return opt,x0_attn,x1_attn
        return opt


In [2]:
from snli.train_utils import SNLI_model, snli_glove_data_module, snli_bert_data_module,SwitchOptim,SNLI_struc_attn_model
from utils import *

seed_torch()


data_module = snli_glove_data_module(128)
Lang = data_module.Lang
embedding_matrix = read_embedding_file(Lang)

creating embedding matrix


In [3]:
model_conf = Struc_Attn_Encoder_conf(Lang,embedding_matrix)
model = Struc_Attn_encoder_snli(model_conf)


In [4]:
for i in data_module.test_dataloader():
    print(i)
    break

[tensor([[  1, 223, 604,  ...,   0,   0,   2],
        [  1, 223, 604,  ...,   0,   0,   2],
        [  1, 223, 604,  ...,   0,   0,   2],
        ...,
        [  1,  16,  59,  ...,   0,   0,   2],
        [  1, 479, 134,  ...,   0,   0,   2],
        [  1, 479, 134,  ...,   0,   0,   2]]), tensor([[   1,    6,  604,  ...,    0,    0,    2],
        [   1,    6,  604,  ...,    0,    0,    2],
        [   1,    4, 1246,  ...,    0,    0,    2],
        ...,
        [   1,   16,  134,  ...,    0,    0,    2],
        [   1,    4,  127,  ...,    0,    0,    2],
        [   1,   16,  134,  ...,    0,    0,    2]]), tensor([2, 1, 0, 2, 1, 0, 1, 2, 0, 2, 1, 0, 1, 1, 0, 0, 1, 2, 1, 0, 2, 0, 1, 2,
        2, 0, 1, 1, 2, 0, 0, 1, 1, 1, 2, 0, 2, 1, 0, 1, 2, 0, 0, 1, 2, 2, 0, 1,
        2, 0, 2, 1, 0, 2, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 0,
        1, 2, 0, 1, 2, 1, 1, 0, 0, 0, 1, 1, 0, 2, 0, 2, 1, 2, 0, 1, 2, 1, 0, 0,
        1, 2, 2, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0, 2, 1,

In [103]:
model(i[0],i[1]).shape

torch.Size([100, 128, 30]) torch.Size([30, 128, 600])
torch.Size([100, 128, 30]) torch.Size([30, 128, 600])


torch.Size([128, 3])

In [4]:
import pytorch_lightning as pl
import torch.nn.functional as F
from utils import *
from datamodule import *
from pytorch_lightning.callbacks import LearningRateLogger
from snli.attn_enc.attn_enc import *
from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.metrics import Accuracy
import pickle
import os
from joblib import Memory
import shutil
import argparse
from lang import *
from snli.train_utils import SNLI_model, snli_glove_data_module, snli_bert_data_module,SwitchOptim
from keys import NEPTUNE_API

conf_kwargs = {
        "batch_size": 128,
        "max_len": 100,
        "embedding_dim": 300,
        "hidden_size" :300,
        "fcs" : 1,
        "r" : 30,
        "num_layers" : 2,
        "dropout" : 0.1,
        "opt_labels" : 3,
        "bidirectional" : True,
        "attention_layer_param" :100,
        "activation" : "tanh",
        "freeze_embedding" : False,
        "gated_embedding_dim" : 300,
        "gated" : False,
        "pool_strategy" : 'max',
        "C":1.
    }

hparams = {
    "optimizer_base":{
        "optim": "adamw",
        "lr": 0.0010039910781394373,
        "scheduler": "const"
        },
    "optimizer_tune":{
        "optim": "sgd",
        "lr": 3e-4,
        "momentum": 0,
        "weight_decay": 0,
        "scheduler": "const"
    },
    "switch_epoch":10,
} 

Lang = data_module.Lang

model_conf = Struc_Attn_Encoder_conf(Lang, embedding_matrix,**conf_kwargs)
model = SNLI_struc_attn_model(Struc_Attn_encoder_snli, model_conf, hparams)

EPOCHS = 6

neptune_logger = NeptuneLogger(
    api_key=NEPTUNE_API,
    project_name="aditya140/SNLI",
    experiment_name="Evaluation",
    tags=["Struc Self Attention"],
)
tensorboard_logger = TensorBoardLogger("lightning_logs")
lr_logger = LearningRateLogger(logging_interval="step")

neptune_logger.experiment.log_metric("epochs", EPOCHS)
trainer = pl.Trainer(
    gpus=1,
    max_epochs=EPOCHS,
    progress_bar_refresh_rate=10,
    profiler=False,
    auto_lr_find=False,
    callbacks=[lr_logger,SwitchOptim()],
    logger=[neptune_logger, tensorboard_logger],
    row_log_interval=2,
    # gradient_clip_val=0.5,
)
trainer.fit(model, data_module)
trainer.test(model, datamodule=data_module)


https://ui.neptune.ai/aditya140/SNLI/e/SNLI-215
NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                    | Params | In sizes | Out sizes
-------------------------------------------------------------------------
0 | model | Struc_Attn_encoder_snli | 27 M   | ?        | ?        


Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment SNLI-215. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: 1e49768c-5024-452e-ae2a-6419158835

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment SNLI-215. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: 1e49768c-5024-452e-ae2a-6419158835

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment SNLI-215. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: 1e49768c-5024-452e-ae2a-6419158835

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment SNLI-215. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: 1e49768c-5024-452e-ae2a-6419158835

Failed to send channel value.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/usr/local/lib/python3.6/dist-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/neptune/internal/backends/hosted_neptune_backend.py", line 565, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchError: Received batch errors sending channels' values to experiment SNLI-215. Cause: Error(code=400, message='X-coordinates must be strictly increasing for channel: 1e49768c-5024-452e-ae2a-6419158835

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7854), 'test_loss': tensor(0.5690, device='cuda:0')}
--------------------------------------------------------------------------------



[{'test_acc': 0.7854099273681641, 'test_loss': 0.5690193772315979}]

In [34]:
a.shape

torch.Size([30, 600, 50])

In [35]:
b= torch.rand((30,1,600))

In [38]:
torch.bmm(b,a).shape.shape

torch.Size([30, 1, 50])

In [46]:
emb = torch.ones((30,128,600))
wt = torch.ones((30,600,50))


In [42]:
torch.bmm(emb.view(30*128,1,600),wt)

RuntimeError: Expected tensor to have size 3840 at dimension 0, but got size 30 for argument #2 'batch2' (while checking arguments for bmm)

In [43]:
F.linear(emb,wt)

RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [45]:
emb.matmul(wt).shape

torch.Size([30, 128, 50])

In [52]:
emb = torch.rand((3,3,2))
wt = torch.ones((3,2,4))

In [62]:
emb[:,0,:].unsqueeze(1).matmul(wt)

tensor([[[0.6342, 0.6342, 0.6342, 0.6342]],

        [[0.8806, 0.8806, 0.8806, 0.8806]],

        [[0.8448, 0.8448, 0.8448, 0.8448]]])

In [54]:
emb

tensor([[[0.4267, 0.2076],
         [0.8026, 0.1468],
         [0.3473, 0.9222]],

        [[0.8667, 0.0139],
         [0.6279, 0.6860],
         [0.0298, 0.9107]],

        [[0.2677, 0.5772],
         [0.7093, 0.7226],
         [0.4398, 0.0881]]])

In [65]:
emb.matmul(wt)[:,0,:].unsqueeze(1)

tensor([[[0.6342, 0.6342, 0.6342, 0.6342]],

        [[0.8806, 0.8806, 0.8806, 0.8806]],

        [[0.8448, 0.8448, 0.8448, 0.8448]]])

In [66]:
wt = torch.nn.Parameter(torch.rand((3,2,4)))

In [72]:
from torch.nn.init import kaiming_uniform_
import math

In [73]:
kaiming_uniform_(wt,a=math.sqrt(5))

Parameter containing:
tensor([[[ 0.0090, -0.2795, -0.0215, -0.1704],
         [-0.0792,  0.0280,  0.3027,  0.3143]],

        [[-0.2606, -0.2643,  0.2702,  0.0258],
         [-0.1872, -0.1068,  0.2531,  0.3323]],

        [[ 0.3368, -0.0465, -0.1596, -0.0386],
         [ 0.0175, -0.1783, -0.1059, -0.2353]]], requires_grad=True)

In [90]:
a = torch.rand([30, 128, 100])

In [94]:
a.permute(1,0,2).flatten(start_dim =1).shape

torch.Size([128, 3000])

In [11]:
att = torch.rand([100, 128, 30]).permute(1,0,2)

In [19]:
(torch.norm(torch.bmm(att,att.transpose(1,2)) - torch.eye(att.size(1)),p='fro')/att.size(0))**2

tensor(4539.1821)